# Imports

In [ ]:
from asapdiscovery.data.schema_v2.ligand import Ligand
from asapdiscovery.data.schema_v2.molfile import MolFileFactory
from tqdm import tqdm
from itertools import product, combinations, combinations_with_replacement
import pandas as pd
from pathlib import Path
from asapdiscovery.data.openeye import oechem
from asapdiscovery.data.schema_v2.fragalysis import FragalysisFactory

In [ ]:
ff = FragalysisFactory(parent_dir="/Users/alexpayne/Scientific_Projects/mers-drug-discovery/sars2-retrospective-analysis/full_frag_prepped_mpro_20230603/")

## this isn't gonna work because the filenames in the new "aligned" directory is different from anything in metadata.csv

# let's try just parsing the sdf

In [ ]:
MolFileFactory?

In [ ]:
mff = MolFileFactory.from_file(filename="/Users/alexpayne/Scientific_Projects/mers-drug-discovery/sars2-retrospective-analysis/full_frag_prepped_mpro_20230603/2022_12_02_fragalysis_correct_bond_orders_220_P_structures.sdf")

In [ ]:
from asapdiscovery.data.schema_v2.ligand import ChemicalRelationship

In [ ]:
tested_dict = {}
for lig1, lig2 in tqdm(combinations(mff.ligands, 2)):
    if lig1.get_chemical_relationship(lig2) not in ChemicalRelationship.DISTINCT:
        print(lig1.compound_name, lig2.compound_name)

In [ ]:
duplicates = [
    ('ALP-POS-4483ae88-4', 'MIK-UNK-78dbf1b8-1'),
    ('BEN-BAS-c2bc0d80-6', 'VLA-UCB-50c39ae8-2'),
    ('BEN-DND-f2e727cd-5', 'MAT-POS-3ccb8ef6-1'),
    ('EDG-MED-0e5afe9d-3', 'PET-UNK-29afea89-2'),
    ('EDG-MED-5d232de5-5', 'MIC-UNK-91acba05-6'),
    ('EDG-MED-5d232de5-7', 'EDG-MED-5d232de5-8'),
    ('EDG-MED-5d232de5-7', 'PET-UNK-c9c1e0d8-4'),
    ('EDG-MED-5d232de5-8', 'PET-UNK-c9c1e0d8-4'),
    ('EDJ-MED-015fb6b4-2', 'MAT-POS-a54ce14d-2'),
    ('EDJ-MED-37aac4bd-4', 'MAT-POS-932d1078-3'),
    ('EDJ-MED-8bb691af-4', 'MAT-POS-c7726e07-5'),
    ('EDJ-MED-976a33d5-1', 'MAT-POS-e48723dc-2'),
    ('MAT-POS-090737b9-1', 'VLA-UCB-50c39ae8-7'),
    ('MAT-POS-7174c657-5', 'MAT-POS-7174c657-6'),
    ('MAT-POS-7174c657-5', 'MAT-POS-a13804f0-4'),
    ('MAT-POS-7174c657-6', 'MAT-POS-a13804f0-4'),
    ('MAT-POS-a13804f0-3', 'RAL-THA-05e671eb-10'),
    ('MIK-ENA-5d9157e9-5', 'MIK-ENA-5d9157e9-6'),
    ('VLA-UCB-29506327-1', 'VLA-UNK-cf7facf1-1')
]

In [ ]:
unique = {item for sublist in duplicates for item in sublist}

In [ ]:
unique_ligs = {lig.compound_name: lig for lig in mff.ligands if lig.compound_name in unique}

In [ ]:
df = pd.DataFrame({"Smiles": [lig.smiles for lig in mff.ligands if lig.compound_name in unique], "Compound_ID": [lig.compound_name for lig in mff.ligands if lig.compound_name in unique]})

In [ ]:
df = pd.DataFrame({"Smiles": [lig1.smiles, lig2.smiles], "Compound_ID": [lig1.compound_name, lig2.compound_name]})

In [ ]:
import mols2grid
from asapdiscovery.data.schema_v2.ligand import Ligand, ChemicalRelationship

In [ ]:
smiles=[]
compound_name=[]
chem_rel = []
lig1_has_stereo = []
lig2_has_stereo = []
for name1, name2 in duplicates:
    lig1 = unique_ligs[name1]
    lig2 = unique_ligs[name2]
    smiles.append(lig1.smiles)
    smiles.append(lig2.smiles)
    compound_name.append(lig1.compound_name)
    compound_name.append(lig2.compound_name)
    chem_rel.append(lig1.get_chemical_relationship(lig2))
    lig1_has_stereo.append(lig1.has_defined_stereo)
    lig2_has_stereo.append(lig2.has_defined_stereo)

In [ ]:
df = pd.DataFrame({"Smiles": smiles, "Compound_ID": compound_name})

In [ ]:
mols2grid.display(df, smiles_col="Smiles")

In [ ]:
rel_df = pd.DataFrame({"Lig1": [lig[0] for lig in duplicates], 
                       "Lig2": [lig[1] for lig in duplicates], 
                       "Chemical_Relationships":chem_rel, 
                       "Lig1 Stereospecific": lig1_has_stereo,
                      "Lig2 Stereospecific": lig2_has_stereo})

In [ ]:
rel_df["Diff Stereospecificity"] = rel_df["Lig1 Stereospecific"] != rel_df["Lig2 Stereospecific"]

In [ ]:
weird = rel_df[rel_df["Diff Stereospecificity"] == False]

In [ ]:
weird

In [ ]:
weird_list = set(weird.Lig1.unique()) | set((weird.Lig2.unique()))

In [ ]:
weird_list

In [ ]:
mols2grid.display(df[df['Compound_ID'].isin(weird_list)], smiles_col="Smiles")

In [ ]:
rel_df.to_csv("20231212_fragalysis_duplicates.csv")